In [ ]:
import os
import base64
import re
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# -------------------------------------------------------
# 1) FUNCIÓN PARA CALCULAR LA CARPETA SEGÚN EL ASUNTO
# -------------------------------------------------------
import os
import re

def carpeta_destino(asunto: str) -> str:
    """
    Extrae un legajo de 5 dígitos del asunto y crea la carpeta donde se guardarán los adjuntos.
    Si no encuentra el legajo, usa 'sin_legajo'.
    """

    # Buscar 5 dígitos consecutivos
    match = re.search(r"\b(\d{5,6})\b", asunto)
    
    if match:
        legajo = match.group(1)
        print(f"  -> Legajo encontrado: {legajo}")
        ruta = os.path.join("emails", legajo)
        os.makedirs(ruta, exist_ok=True)
        return ruta
    else:
        return None 


# 2) AUTENTICACIÓN
# -------------------------------------------------------
def get_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

# -------------------------------------------------------
# 3) DESCARGAR ADJUNTOS (SOLO IMÁGENES)
# -------------------------------------------------------
def bajar_adjuntos(msg_id, service, carpeta_out):
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    print(f"  -> Bajando adjuntos del email ID: {msg_id} a la carpeta: {carpeta_out}")
    parts = msg.get("payload", {}).get("parts", [])
    if not parts:
        return

    for part in parts:
        filename = part.get("filename")
        mime = part.get("mimeType", "")

        # Solo imágenes
        if not filename or not mime.startswith("image/"):
            continue

        body = part.get("body", {})
        att_id = body.get("attachmentId")

        if not att_id:
            continue

        # Obtener datos del adjunto
        attachment = service.users().messages().attachments().get( userId='me', messageId=msg_id, id=att_id ).execute()

        data = attachment.get('data')
        file_data = base64.urlsafe_b64decode(data.encode())

        # Guardar archivo
        filepath = os.path.join(carpeta_out, filename)
        with open(filepath, "wb") as f:
            f.write(file_data)

        print(f"Imagen guardada: {filepath}")

# -------------------------------------------------------
# 4) PROCESAR TODOS LOS EMAILS
# -------------------------------------------------------
def bajar_todo():
    print("|-- Iniciando proceso de descarga de adjuntos --|")
    service = get_service()

    # Obtenemos solo los mensajes de la bandeja de entrada
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],  # Solo bandeja de entrada
        maxResults=500  # podés subirlo
    ).execute()

    mensajes = results.get('messages', [])
    print(f"Emails encontrados en INBOX: {len(mensajes)}")

    for m in mensajes:
        msg = service.users().messages().get( userId='me', id=m['id'], format="metadata", metadataHeaders=["Subject"] ).execute()

        headers = msg.get("payload", {}).get("headers", [])
        asunto = "sin asunto"

        for h in headers:
            if h["name"] == "Subject":
                asunto = h["value"]
                break

        # calcular carpeta por asunto
        print(f"Procesando email ID: {m['id']} Asunto: {asunto}")
        if carpeta := carpeta_destino(asunto):    

            bajar_adjuntos(m["id"], service, carpeta)
            print(carpeta)
            pass

    print("|-- Proceso finalizado --|")

# -------------------------------------------------------
# Ejecutar el proceso:
# bajar_todo()


|-- Iniciando proceso de descarga de adjuntos --|
Emails encontrados en INBOX: 1
Procesando email ID: 19a816da82719279 Asunto: 612000
  -> Legajo encontrado: 612000
  -> Bajando adjuntos del email ID: 19a816da82719279 a la carpeta: emails/612000
Imagen guardada: emails/612000/p1.png
Imagen guardada: emails/612000/p5.png
Imagen guardada: emails/612000/p3.png
Imagen guardada: emails/612000/p4.png
Imagen guardada: emails/612000/p2.png
|-- Proceso finalizado --|


In [ ]:
from pathlib import Path
from typing import Iterable

from PIL import Image

SUPPORTED_IMAGE_EXTENSIONS = {
.    ".png",
.    ".jpeg",
.    ".jpg",
.    ".bmp",
.    ".gif",
.    ".tiff",
.    ".webp",
.    ".heic",
.    ".heif",
.}

def iter_image_paths(root_path: Path) -> Iterable[Path]:
    """Yield image files inside root_path recursively."""
    for entry in root_path.rglob("*"):
        if entry.is_file() and entry.suffix.lower() in SUPPORTED_IMAGE_EXTENSIONS:
            yield entry

def convert_images_to_jpg(
    root_dir: str | Path,
    quality: int = 90,
    overwrite: bool = False,
    delete_original: bool = False,
) -> dict:
    """Convert every image under root_dir to JPEG, optionally overwriting/deleting originals."""
    root_path = Path(root_dir).expanduser().resolve()
    if not root_path.exists():
        raise FileNotFoundError(f"La carpeta {root_path} no existe")

    converted = 0
    skipped = 0
    errors = 0

    for image_path in iter_image_paths(root_path):
        suffix = image_path.suffix.lower()

        if suffix in {".jpg", ".jpeg"} and not overwrite:
            skipped += 1
            continue

        target_path = image_path.with_suffix(".jpg")
        if target_path.exists() and not overwrite:
            skipped += 1
            continue

        try:
            with Image.open(image_path) as img:
                rgb_img = img.convert("RGB")
                rgb_img.save(target_path, "JPEG", quality=quality, optimize=True)

            converted += 1
            print(f"Convertido: {image_path} -> {target_path}")

            if delete_original and target_path != image_path:
                image_path.unlink()

        except Exception as exc:
            errors += 1
            print(f"[ERROR] No se pudo convertir {image_path}: {exc}")

    resumen = {"convertidos": converted, "saltados": skipped, "errores": errors}
    print(f"Resumen: {resumen}")
    return resumen

CARPETA_IMAGENES = Path("emails")
if CARPETA_IMAGENES.exists():
    convert_images_to_jpg(CARPETA_IMAGENES, quality=90, overwrite=False, delete_original=False)
else:
    print(f"La carpeta {CARPETA_IMAGENES} aún no existe. Crear la carpeta o modificar CARPETA_IMAGENES.")